<a href="https://colab.research.google.com/github/COOLSTORM36/InvoiceExtractor/blob/main/MarkItDown!.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Prerequisite

In [ ]:
!pip install markitdown langchain-community langchain langchain_openai openai

In [ ]:
from markitdown import MarkItDown
import openai
from google.colab import userdata
from langchain import LLMChain
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

openai_api = userdata.get('OPENAI_API_KEY')
openai.api_key = openai_api

client = openai
md = MarkItDown(llm_client=client, llm_model="gpt-4o")

# MarkItDown Initialize

In [ ]:
def md_convert(file_path):
    try:
        mdresult = md.convert(file_path)
        # print(mdresult.text_content)
        return mdresult.text_content
    except Exception:
        print(f"An error occurred:")
        return None

# Parse invoice method

In [ ]:
def parse_invoice(text):
    # Create an instance of InvoiceSchema to get format instructions

    # Step 1: Define the response schema
    response_schemas = [
        ResponseSchema(name="Invoice Number", description="The unique number of the invoice."),
        ResponseSchema(name="Date", description="The date when the invoice was issued."),
        ResponseSchema(name="Total Amount", description="The total amount listed on the invoice."),
        ResponseSchema(name="Vendor Name", description="The name of the vendor who issued the invoice.")
    ]

    # Step 2: Create a StructuredOutputParser
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

    # Step 3: Generate format instructions for the prompt
    format_instructions = output_parser.get_format_instructions()

    # Create a prompt template for parsing invoices
    prompt_template = PromptTemplate(
        input_variables=["invoice_text", "format_instructions"],
        template=(
            "Extract the following information from the invoice text below and return it in JSON format, if the info is missing, leave as blank:\n"
            "{format_instructions}\n\n"
            "Invoice Text:\n{invoice_text}"
        )
    )

    # Create an LLM chain with the prompt template and parser

    chain = LLMChain(
        llm=ChatOpenAI(openai_api_key=openai_api, temperature=0, model_name="gpt-4o"),
        prompt=prompt_template,
        output_parser=output_parser,
    )

    # Run the chain with the invoice text and format instructions
    result = chain.run(invoice_text=text, format_instructions=format_instructions)
    return result

In [ ]:
filepath = ""
parsed_data = parse_invoice(md_convert(filepath))
print(parsed_data)

In [ ]:
parsed_data

In [ ]:
parsed_data.get('Total Amount')